# Generate Synthetic Data and Evaluate the Dataset

Contoso Zoo is developing an AI-powered smart guide app to enhance visitors' experiences. The app aims to answer questions about wildlife. To evaluate the responses from the app, we need to create a comprehensive synthetic question-answer dataset that covers various aspects of wildlife, including animal behavior, habitat preferences, and dietary needs. Once we have the synthetic dataset, we'll need to evaluate the model's responses. <br>

In this exercise, you will generate a synthetic question-answer dataset for the Contoso Zoo app. This dataset will not only include standard question-answer pairs but also incorporate conversation starters to simulate contextually relevant interactions. These conversation starters will allow for pre-specified, repeatable user turns in a conversation, which is crucial for evaluating the AI's performance across different scenarios.

Afterwards, we'll evaluate the model's responsese for fluency, coherence, and harm (i.e. the Risk & Safety evaluators).


## Add environment variables to the .env file

In the root of the **Evaluation and Data Generation Workshop** folder is an `.env` file. Within the `.env` file, fill in the values for the environment variables. You can locate the values for each environment variable in the following locations of the [Azure AI Foundry](https://ai.azure.com) portal:

- `AZURE_SUBSCRIPTION_ID` - On the **Overview** page of your project within **Project details**.
- `AZURE_AI_PROJECT_NAME` - At the top of the **Overview** page for your project.
- `AZURE_OPENAI_RESOURCE_GROUP` - On the **Overview** page of the **Management Center** within **Project properties**.
- `AZURE_OPENAI_SERVICE` - On the **Overview** page of your project in the **Included capabilities** tab for **Azure OpenAI Service**.
- `AZURE_OPENAI_API_VERSION` - On the [API version lifecycle](https://learn.microsoft.com/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release) webpage within the **Latest GA API release** section.
- `AZURE_OPENAI_ENDPOINT` - On the **Details** tab of your model deployment within **Endpoint** (i.e. **Target URI**)
- `AZURE_OPENAI_DEPLOYMENT_NAME` -  On the **Details** tab of your model deployment within **Deployment info**.

# Sign in to Azure

As a security best practice, we'll use [keyless authentication](https://learn.microsoft.com/azure/developer/ai/keyless-connections?tabs=csharp%2Cazure-cli) to authenticate to Azure OpenAI with Microsoft Entra ID. Before you can do so, you'll first need to install the **Azure CLI** per the [installation instructions](https://learn.microsoft.com/cli/azure/install-azure-cli) for your operating system.

Next, open a terminal and run `az login` to sign in to your Azure account.

## Install packages

To begin, we'll install the packages that are necessary for completing the simulation. The `Simulator` class is within the `simulator` package of the Azure AI Evaluation SDK. With security in mind, we'll authenticate with Azure services using `azure-identity`. And finally, since we'll be interacting with OpenAI's services via Azure, the `openai` package will come in handy!

In [1]:
%pip install azure-ai-evaluation azure-identity openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Access the environment variables.

We'll import `os` and `load_dotenv` so that you can access the environment variables.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Setup keyless authentication

Rather than hardcode your **key**, we'll use a keyless connection with Azure OpenAI.

In [3]:
import azure.identity

credential = azure.identity.DefaultAzureCredential()
token_provider = azure.identity.get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

token = token_provider()

## Import packages

We'll now import the `Simulator` class alongside several additional packages and modules that'll be necessary for running the simulation.

In [4]:
import json
from azure.ai.evaluation.simulator import Simulator
from typing import List, Dict, Any, Optional
from openai import AzureOpenAI

[INFO] Could not import AIAgentConverter. Please install the dependency with `pip install azure-ai-projects`.
[INFO] Could not import SKAgentConverter. Please install the dependency with `pip install semantic-kernel`.


## Configure the model_config

The `model_config` is necessary as it's a required parameter when creating an instance of the `Simulator` class. Let's configure the `model_config` with the following:

- Azure OpenAI endpoint
- Azure OpenAI API key
- Azure deployment

In [5]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": token,
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
}

should_cleanup: bool = False

## Create an instance of the Simulator class

We now need to create an instance of the `Simulator` class which will later be used to run the simulator. We'll need to initialize the `simulator` object with the `model_config`.

In [6]:
simulator = Simulator(model_config)

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## Create a call to your application

We'll now create a function that'll create a call to your application which should take a user query and return a response.

The call to the AI model includes parameters which specifies the model, messages, and additional settings which impact the model's response.

We'll later send a query to the AI model and retrieve the repsonse. Once the repsonse is retrieved, we'll convert the completion object to a dictionary and extract the message before returning the message content.

In [7]:
def call_to_your_ai_application(query: str) -> str:
    # logic to call your application
    # use a try except block to catch any errors
    deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        api_key= token,
    )
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )
    message = completion.to_dict()["choices"][0]["message"]
    # change this to return the response from your application
    return message["content"]

## Create a callback

We'll now create a `callback` which will handle incoming messages, process them using the app, and format the response to follow the OpenAI chat protocol format.

The `callback` function will extract the latest message from the conversation history, reset the context, and then call the app to get a response based on the latest message.

Afterwards, the response will be formatted to follow the OpenAI chat protocol format, and then appended to the conversation history. We'll return the updated conversation history.

In [8]:
async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_to_your_ai_application(query)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {
            "citations": None,
        },
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}


## Create conversation starters

When defining the conversation starters, we want to provide examples that are relevant to a real-world use-case. Here we provide 2 conversation starters for two separate conversations.

The first conversation simulates a person discussing lions with the app. The second conversation simulates a parent inquiring about birds with the app. Both conversations are stored within a `conversations_turns` list.

**Note**: Later when you run the simulator, if you consistently exceed quota, remove one of the simulations from the `conversation_turns` list and run the subsequent cells once more.

In [9]:
conversation_turns = [
    # simulation 1
    [
        "I'm standing in front of the lion enclosure, and I've never seen a real lion before!",  # simulator conversation starter (turn 1)
        "Wow, the lion is much bigger than I expected! How much do they typically weigh?",  # conversation turn 2,
        "I notice the lion is just lying there. Do they sleep a lot? What do they do most of the day?",  # conversation turn 3
    ],
    # simulation 2
    [
        "My child is fascinated by the colorful birds in the aviary. She keeps asking about their feathers.",
        "My daughter wants to know why some birds have such bright colors while others are more dull. Can you explain?",
        "She's now wondering if birds can change the color of their feathers like chameleons change their skin color.",
    ],
]

## Create a call to the simulator

Let's now create a call to run the simulation and process the conversation turns.

In [10]:
outputs = await simulator(
    target=callback,
    conversation_turns=conversation_turns,
    max_conversation_turns=3,
)

Simulating with predefined conversation turns: 100%|████████████| 6/6 [00:26<00:00,  4.47s/messages]


## Convert the simulation conversation pairs into jsonl

We'll now need to convert the simulated conversation into query and response pairs and place into a `.jsonl` file. A `dataset_converter` module is provided which completes the conversion using the `convert_conversations_to_jsonl()` method.

In [11]:
from dataset_converter import convert_conversations_to_jsonl

json_data = json.dumps(outputs)
jsonl_output = convert_conversations_to_jsonl(json_data)

# Write to a file
with open('zoo_conversations.jsonl', 'w', encoding='utf-8') as f:
    f.write(jsonl_output)

print("File has been saved as 'zoo_conversations.jsonl'")

File has been saved as 'zoo_conversations.jsonl'


## Set the path of the dataset

We'll now set the path of the dataset that we'll use for the evalutation. The dataset is within the `zoo_conversations.jsonl` file and consists of the following values for each row of data:

- query
- response

In [12]:
path = "zoo_conversations.jsonl"

## Configure the Azure AI project

We'll now configure the Azure AI project with the following:

- Azure project name
- Resource group name
- Subscription ID

In [13]:
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_OPENAI_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_AI_PROJECT_NAME"),
}

## Create an instance of the evaluators

Let's now create an instance of the evaluators. The performance & quality evaluators (i.e. coherence and fluency) only required that we pass in the `model_config`. However, the risk and safety evaluators (i.e. violence, hate and unfairness, sexual, and self-harm) required both the `azure_ai_project` and `credential`. 

In [14]:
from azure.ai.evaluation import evaluate, CoherenceEvaluator, FluencyEvaluator, ViolenceEvaluator, HateUnfairnessEvaluator, SexualEvaluator, SelfHarmEvaluator
from azure.identity import DefaultAzureCredential 

coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class HateUnfairnessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SexualEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SelfHarmEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## Create the call to evaluate the dataset

We can run an evaluation for a dataset with the `evaluate` function and include our list of evaluators. We must also ensure that the `evaluator_config` is set with the appropriate parameters and values for the `query` and `response`.

Since we want to track the evaluation results in our Azure AI project, we'll need to include the Azure AI Foundry project information.

In [ ]:
result = evaluate(
    data=path,
    evaluators={
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "violence": violence_eval,
        "hate_unfairness": hateunfairness_eval,
        "sexual": sexual_eval,
        "self_harm": selfharm_eval
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    },
    azure_ai_project = azure_ai_project
)

[2025-07-03 16:22:20 +0000][promptflow][WARNING] - Found existing /workspaces/RAI-workshops/Evaluation and Data Generation Workshop/flow.flex.yaml, will not respect it in runtime.
[2025-07-03 16:22:20 +0000][promptflow][WARNING] - Found existing /workspaces/RAI-workshops/Evaluation and Data Generation Workshop/flow.flex.yaml, will not respect it in runtime.
[2025-07-03 16:22:20 +0000][promptflow][WARNING] - Found existing /workspaces/RAI-workshops/Evaluation and Data Generation Workshop/flow.flex.yaml, will not respect it in runtime.
[2025-07-03 16:22:20 +0000][promptflow][WARNING] - Found existing /workspaces/RAI-workshops/Evaluation and Data Generation Workshop/flow.flex.yaml, will not respect it in runtime.
[2025-07-03 16:22:20 +0000][promptflow][WARNING] - Found existing /workspaces/RAI-workshops/Evaluation and Data Generation Workshop/flow.flex.yaml, will not respect it in runtime.
[2025-07-03 16:22:20 +0000][promptflow][WARNING] - Found existing /workspaces/RAI-workshops/Evaluati

2025-07-03 16:22:20 +0000   15978 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-07-03 16:22:23 +0000   15978 execution.bulk     INFO     Finished 1 / 6 lines.
2025-07-03 16:22:23 +0000   15978 execution.bulk     INFO     Average execution time for completed lines: 3.13 seconds. Estimated time for incomplete lines: 15.65 seconds.
2025-07-03 16:22:23 +0000   15978 execution.bulk     INFO     Finished 2 / 6 lines.
2025-07-03 16:22:23 +0000   15978 execution.bulk     INFO     Average execution time for completed lines: 1.58 seconds. Estimated time for incomplete lines: 6.32 seconds.
2025-07-03 16:22:24 +0000   15978 execution.bulk     INFO     Finished 3 / 6 lines.
2025-07-03 16:22:24 +0000   15978 execution.bulk     INFO     Average execution time for completed lines: 1.16 seconds. Estimated time for incomplete lines: 3.48 seconds.
2025-07-03 16:22:24 +0000   15978 execution.bulk     INFO     Finished 4 / 6 lines.
2025

https://ai.azure.com/build/evaluation/1a1a072a-14d3-485f-862d-f2140b6b831d?wsid=/subscriptions/973bd745-2c33-42c1-9ec6-b11a0a7f4aba/resourceGroups/rg-franciscoruizrivas-8637_ai/providers/Microsoft.MachineLearningServices/workspaces/franciscoruizrivas-4283
[{"name": "Any", "type": "_AnyMeta", "fullType": "typing._AnyMeta"}, {"name": "AzureOpenAI", "type": "type", "fullType": "type"}, {"name": "CoherenceEvaluator", "type": "ABCMeta", "fullType": "abc.ABCMeta"}, {"name": "DefaultAzureCredential", "type": "type", "fullType": "type"}, {"name": "Dict", "type": "_SpecialGenericAlias", "fullType": "typing._SpecialGenericAlias"}, {"name": "FluencyEvaluator", "type": "ABCMeta", "fullType": "abc.ABCMeta"}, {"name": "HateUnfairnessEvaluator", "type": "ABCMeta", "fullType": "abc.ABCMeta"}, {"name": "List", "type": "_SpecialGenericAlias", "fullType": "typing._SpecialGenericAlias"}, {"name": "Optional", "type": "_SpecialForm", "fullType": "typing._SpecialForm"}, {"name": "SelfHarmEvaluator", "type": 

## View the results in Azure AI Foundry

Now that the evaluation is complete, you can navigate to the **Evaluation** section of the Azure AI Foundry to view the results. Alternatively, you can output a link to the evaluation location using the `studio_url` returned from running `evaluate`.

In [16]:
print(result['studio_url'])